In [1]:
import input_data
mnist = input_data.read_data_sets("/tmp/data/", one_hot = True)
import tensorflow as tf

Succesfully downloaded train-images-idx3-ubyte.gz 9912422 bytes.
Extracting /tmp/data/train-images-idx3-ubyte.gz


/usr/lib/python3.4/gzip.py:372: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  chunk = self.extrabuf[offset: offset + size]
/home/ubuntu/pynb/input_data.py:42: VisibleDeprecationWarning: converting an array with ndim > 0 to an index will result in an error in the future
  data = data.reshape(num_images, rows, cols, 1)


Succesfully downloaded train-labels-idx1-ubyte.gz 28881 bytes.
Extracting /tmp/data/train-labels-idx1-ubyte.gz
Succesfully downloaded t10k-images-idx3-ubyte.gz 1648877 bytes.
Extracting /tmp/data/t10k-images-idx3-ubyte.gz
Succesfully downloaded t10k-labels-idx1-ubyte.gz 4542 bytes.
Extracting /tmp/data/t10k-labels-idx1-ubyte.gz


In [2]:
learning_rate = 0.01
training_iteration = 30
batch_size = 100
display_step = 2

In [3]:
x = tf.placeholder("float", [None, 784])
y = tf.placeholder("float", [None, 10])

In [5]:
W = tf.Variable(tf.zeros([784, 10]))
b = tf.Variable(tf.zeros([10]))

In [6]:
with tf.name_scope("Wx_b") as scope:
    model = tf.nn.softmax(tf.matmul(x, W) + b)

In [7]:
w_h = tf.histogram_summary("weigths", W)
b_h = tf.histogram_summary("biases", b)

Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.
Instructions for updating:
Please switch to tf.summary.histogram. Note that tf.summary.histogram uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on their scope.


In [8]:
with tf.name_scope("cost_function") as scope:
    cost_function = -tf.reduce_sum(y * tf.log(model))
    tf.scalar_summary("cost_funtion", cost_function)

Instructions for updating:
Please switch to tf.summary.scalar. Note that tf.summary.scalar uses the node name instead of the tag. This means that TensorFlow will automatically de-duplicate summary names based on the scope they are created in. Also, passing a tensor or list of tags to a scalar summary op is no longer supported.


In [9]:
with tf.name_scope("train") as scope:
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost_function)

In [10]:
init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [11]:
merged_summary_op = tf.merge_all_summaries()

Instructions for updating:
Please switch to tf.summary.merge_all.
Instructions for updating:
Please switch to tf.summary.merge.


In [14]:
with tf.Session() as sess:
    sess.run(init)
    
    summary_writer = tf.train.SummaryWriter('~/', graph_def= sess.graph_def)
    
    for iteration in range(training_iteration):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x : batch_xs, y:batch_ys})
            avg_cost += sess.run(cost_function, feed_dict={x: batch_xs, y: batch_ys})/total_batch
            
            summary_str = sess.run(merged_summary_op, feed_dict={x: batch_xs, y: batch_ys})
            summary_writer.add_summary(summary_str, iteration*total_batch + 1)
            
           
        if iteration % display_step == 0:
            print("iteration:", '%04d' %(iteration + 1), "cost=","{:.9f}".format(avg_cost))
    
    print("Tuning completed!")
    
    predictions = tf.equal(tf.argmax(model, 1), tf.argmax(y, 1))
    
    accuracy = tf.reduce_mean(tf.cast(predictions, "float"))
    print("Accuracy:", accuracy.eval({x: mnist.test.images, y: mnist.test.labels}))
            

Instructions for updating:
Please switch to tf.summary.FileWriter. The interface and behavior is the same; this is just a rename.
iteration: 0001 cost= 29.860459408
iteration: 0003 cost= 21.044074940
iteration: 0005 cost= 20.167027329
iteration: 0007 cost= 19.629600663
iteration: 0009 cost= 19.217226710
iteration: 0011 cost= 19.193944042
iteration: 0013 cost= 18.886636833
iteration: 0015 cost= 18.861951441
iteration: 0017 cost= 18.708302399
iteration: 0019 cost= 18.500450553
iteration: 0021 cost= 18.469158167
iteration: 0023 cost= 18.330104531
iteration: 0025 cost= 18.352073899
iteration: 0027 cost= 18.181169332
iteration: 0029 cost= 18.135102044
Tuning completed!
Accuracy: 0.9223
